# Topic modeling 
Objective : identify topics emerging from comments

##  Required packages

In [28]:
#import warnings
#warnings.filterwarnings('ignore')

import pandas as pd
import gensim

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models

from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer



## Load data

In [29]:
# load the data
df = pd.read_csv("data_for_tpm.csv")

# check nan values
print(df.isna().sum())

print(df.info())

Unnamed: 0            0
comment               0
rating                0
comment_cleaned       0
comment_lemmatized    0
comment_stem          0
stem_len              0
comment_topics        0
cmt_length            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22462 entries, 0 to 22461
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          22462 non-null  int64 
 1   comment             22462 non-null  object
 2   rating              22462 non-null  int64 
 3   comment_cleaned     22462 non-null  object
 4   comment_lemmatized  22462 non-null  object
 5   comment_stem        22462 non-null  object
 6   stem_len            22462 non-null  int64 
 7   comment_topics      22462 non-null  object
 8   cmt_length          22462 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.5+ MB
None


In [30]:
# split data into training and test
from sklearn.model_selection import train_test_split
X = df["comment_topics"]
y= df["rating"]

doc_train, doc_test, y_train, y_test = train_test_split(X, y, 
                                                        stratify=y, 
                                                        test_size=0.25)

/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [31]:
# Join all the comment_topics into a single doc : doc_to_topics
doc_to_topics = [cmt.split() for cmt in doc_train]

# put data into a gensim dictionnary and then convert it to a Bag of Words, 
# that is couple word/"frequency of occurences"

dictionary = gensim.corpora.Dictionary(doc_to_topics)

dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=1000)

bow_corpus = [dictionary.doc2bow(doc) for doc in doc_to_topics]


### LDA implementation

In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics = 10, 
                                       id2word = dictionary, 
                                       passes = 1000)

In [19]:
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

Topic: 0 -> Words: 0.268*"prix" + 0.084*"rapport" + 0.029*"frais" + 0.028*"paiement" + 0.025*"linge" + 0.019*"meuble" + 0.019*"maison" + 0.018*"réduction" + 0.018*"correct" + 0.017*"port"
Topic: 1 -> Words: 0.061*"efficace" + 0.041*"housse" + 0.039*"couette" + 0.029*"draps" + 0.025*"belle" + 0.024*"satisfait" + 0.024*"redire" + 0.023*"acheté" + 0.021*"aussi" + 0.019*"sérieux"
Topic: 2 -> Words: 0.059*"redoute" + 0.029*"articles" + 0.020*"retour" + 0.019*"site" + 0.015*"trouve" + 0.015*"dommage" + 0.014*"vêtements" + 0.014*"pratique" + 0.012*"autres" + 0.011*"passé"
Topic: 3 -> Words: 0.184*"conforme" + 0.073*"rapidement" + 0.061*"expédition" + 0.059*"article" + 0.045*"description" + 0.038*"attentes" + 0.026*"ravie" + 0.025*"réception" + 0.024*"colis" + 0.022*"vite"
Topic: 4 -> Words: 0.066*"taille" + 0.039*"correspond" + 0.037*"couleur" + 0.029*"site" + 0.027*"commandé" + 0.027*"photo" + 0.024*"petit" + 0.020*"parfaite" + 0.020*"article" + 0.020*"agréable"
Topic: 5 -> Words: 0.077*"ser

### Model coherence

Les mesures de cohérence évaluent le degré de similitude sémantique entre les mots les mieux notés dans le topics. Ces mesures aident à faire la distinction entre les topics sémantiquement interprétables et les topics dû à des inférences statistiques.

Pour un bon modèle LDA la cohérence doit être comprise entre 0.4 et 0.7 au-delà et en dessous le modèle est très probablement erroné. La cohérence pour un modèle LDA est calculée en procédant aux étapes suivantes :

- Segmentation : création de paires de mots à partir de sous-ensembles ;
- Calcul des probabilités : calcul probabilité d’occurrence d’un mot ;
- Mesure de confirmation : vérification « dans quelle mesure » un sous-ensemble de mots supporte un autre sous-ensemble de mots dans chaque paire ;
- Agrégation : agrégation de toutes les valeurs calculées à l’étape précédente en une seule valeur qui est notre score final de cohérence de sujet.

Source: La revue IA 
https://larevueia.fr/latent-dirichlet-allocation-topic-modeling-en-python/

In [20]:
coherence_model_lda = CoherenceModel(model=lda_model, 
                                     texts=doc_to_topics, 
                                     dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.41560903153425494


xxx

### Save Results

In [21]:
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [22]:
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 
                               2: "Topic 3", 3: "Topic 4", 
                               4: "Topic 5", 5: "Topic 6", 
                               6: "Topic 7", 7: "Topic 8", 
                               8: "Topic 9", 9: "Topic 10"})

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.268, prix)","(0.084, rapport)","(0.029, frais)","(0.028, paiement)","(0.025, linge)","(0.019, meuble)","(0.019, maison)","(0.018, réduction)","(0.018, correct)","(0.017, port)"
Topic 2,"(0.061, efficace)","(0.041, housse)","(0.039, couette)","(0.029, draps)","(0.025, belle)","(0.024, satisfait)","(0.024, redire)","(0.023, acheté)","(0.021, aussi)","(0.019, sérieux)"
Topic 3,"(0.059, redoute)","(0.029, articles)","(0.020, retour)","(0.019, site)","(0.015, trouve)","(0.015, dommage)","(0.014, vêtements)","(0.014, pratique)","(0.012, autres)","(0.011, passé)"
Topic 4,"(0.184, conforme)","(0.073, rapidement)","(0.061, expédition)","(0.059, article)","(0.045, description)","(0.038, attentes)","(0.026, ravie)","(0.025, réception)","(0.024, colis)","(0.022, vite)"
Topic 5,"(0.066, taille)","(0.039, correspond)","(0.037, couleur)","(0.029, site)","(0.027, commandé)","(0.027, photo)","(0.024, petit)","(0.020, parfaite)","(0.020, article)","(0.020, agréable)"
Topic 6,"(0.077, service)","(0.040, temps)","(0.032, client)","(0.031, colis)","(0.028, date)","(0.025, redoute)","(0.022, prévue)","(0.020, jours)","(0.017, prévu)","(0.016, heure)"
Topic 7,"(0.133, choix)","(0.057, site)","(0.056, produits)","(0.046, redoute)","(0.036, articles)","(0.030, depuis)","(0.026, facilité)","(0.022, achats)","(0.020, grand)","(0.019, cliente)"
Topic 8,"(0.162, envoi)","(0.084, retour)","(0.074, article)","(0.055, facilité)","(0.049, achat)","(0.048, problème)","(0.038, contente)","(0.036, efficacité)","(0.034, apprécié)","(0.022, aucun)"
Topic 9,"(0.071, colis)","(0.058, relais)","(0.040, point)","(0.031, reçu)","(0.017, commandé)","(0.014, domicile)","(0.014, livré)","(0.012, lendemain)","(0.012, article)","(0.010, bonjour)"
Topic 10,"(0.157, produits)","(0.122, articles)","(0.081, délai)","(0.077, délais)","(0.062, conformes)","(0.037, suivi)","(0.030, respecté)","(0.025, commandés)","(0.023, respect)","(0.022, ultra)"


### Results Visualization

In [23]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.170621 -0.096106       1        1  16.198297
2      0.100274 -0.188214       2        1  13.518716
6      0.307934 -0.031071       3        1  11.645209
4     -0.025314  0.002962       4        1  10.218875
5     -0.130480 -0.035458       5        1   9.818141
9      0.275811  0.256080       6        1   9.190425
3     -0.190937  0.281837       7        1   8.630263
1     -0.066722 -0.120583       8        1   7.417085
7     -0.127086  0.100410       9        1   6.816398
0      0.027141 -0.169857      10        1   6.546591, topic_info=             Term         Freq        Total Category  logprob  loglift
46           prix  1346.000000  1346.000000  Default  30.0000  30.0000
48       conforme  1216.000000  1216.000000  Default  29.0000  29.0000
36       produits  1608.000000  1608.000000  Default  28.0000  28.0000
45          choix  1190.000000  1190.000000  Default  27.0000  27.0000
61          envoi   849.000000   849.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
748  informations    62.207800    90.237397  Topic10  -4.3919   2.3543
326        soldes    46.762430    70.030476  Topic10  -4.6773   2.3224
823         coton    45.437899    77.404460  Topic10  -4.7060   2.1935
118         assez    51.945050   145.046453  Topic10  -4.5722   1.6994
171        panier    31.027868    47.096363  Topic10  -5.0875   2.3089

[521 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
589       3  0.955808   abordable
859      10  0.949467  abordables
731       1  0.984356       abîmé
976      10  0.955446  accessible
229       5  0.986408     accueil
...     ...       ...         ...
401      10  0.975008      élevés
678       8  0.983423       épais
81        1  0.181710        état
81        7  0.817696        état
562       9  0.947168    éventuel

[710 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 3, 7, 5, 6, 10, 4, 2, 8, 1])

In [25]:
df.head()

,Unnamed: 0,comment,rating,comment_cleaned,comment_lemmatized,comment_stem,stem_len,comment_topics,cmt_length
0,0,"Facilte, sollicitations dosées...qualité..on p...",4,facilte sollicitations dosées qualité pourrait...,facilte sollicitation doser qualité pouvoir ce...,facilt sollicit dos qualit pouvoir cepend amél...,558,facilte sollicitations dosées qualité pourrait...,642
1,1,Livraison impeccable avec possibilité de prend...,4,livraison impeccable possibilité prendre samed...,livraison impeccable possibilité prendre samed...,livraison impecc possibil prendr samed matin b...,461,livraison impeccable possibilité prendre samed...,488
2,2,Achat d'un canapé La Redoute intérieurs le 6/9...,4,achat canapé redoute intérieurs site redoute a...,achat canapé redoute intérieur site redoute ab...,achat canap redout intérieur sit redout abord ...,426,achat canapé redoute intérieurs site redoute a...,459
3,3,je commande depuis longtemps chez la redoute; ...,4,commande depuis longtemps redoute appréciais c...,commande depuis longtemps redoute appréciai ch...,command depuis longtemp redout appréci choix r...,370,commande depuis longtemps redoute appréciais c...,417
4,4,"Comme d'habitude, aucun souci et rapidité d'ex...",5,habitude aucun souci rapidité expédition colis...,habitude aucun souci rapidité expédition colis...,habitud aucun souc rapid expédit colis bravo d...,366,habitude aucun souci rapidité expédition colis...,407


### Topic modeling on comments text cleaned and stemmed

In [32]:
# split data into training and test
from sklearn.model_selection import train_test_split
X = df["comment_stem"]
y= df["rating"]

doc_train, doc_test, y_train, y_test = train_test_split(X, y, 
                                                        stratify=y, 
                                                        test_size=0.25)

# Join all the comment_topics into a single doc : doc_to_topics
doc_to_topics = [cmt.split() for cmt in doc_train]

# put data into a gensim dictionnary and then convert it to a Bag of Words, 
# that is couple word/"frequency of occurences"

dictionary = gensim.corpora.Dictionary(doc_to_topics)

dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=1000)

bow_corpus = [dictionary.doc2bow(doc) for doc in doc_to_topics]

# train LDA model
lda_model2 = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics = 10, 
                                       id2word = dictionary, 
                                       passes = 1000)

# Model coherence
coherence_model_lda = CoherenceModel(model=lda_model2, 
                                     texts=doc_to_topics, 
                                     dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

# save result
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

# save results into a dataframe
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 
                               2: "Topic 3", 3: "Topic 4", 
                               4: "Topic 5", 5: "Topic 6", 
                               6: "Topic 7", 7: "Topic 8", 
                               8: "Topic 9", 9: "Topic 10"})

# visualize results
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model2, bow_corpus, dictionary)

/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/dkcentral/anaconda3/envs/deepl_env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Coherence Score:  0.5083791059590385


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.169743  0.014320       1        1  18.529960
7     -0.129835  0.079088       2        1  16.624449
2     -0.145231 -0.115282       3        1   9.717735
5      0.133606 -0.035787       4        1   9.590937
9     -0.030056  0.188996       5        1   9.128198
0     -0.061006 -0.332002       6        1   8.619442
8     -0.153246 -0.026724       7        1   7.608992
6      0.235666  0.069779       8        1   7.390299
4     -0.028342  0.237044       9        1   7.299025
3      0.348188 -0.079432      10        1   5.490962, topic_info=         Term         Freq        Total Category  logprob  loglift
43        del  1293.000000  1293.000000  Default  30.0000  30.0000
64       bien  1722.000000  1722.000000  Default  29.0000  29.0000
5       envoi  1118.000000  1118.000000  Default  28.0000  28.0000
142      prix  1200.000000  1200.000000  Default  27.0000  27.0000
49      facil  1147.000000  1147.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
56    produir   175.869282  1209.524501  Topic10  -3.3418   0.9738
447       dat    40.420700   288.450924  Topic10  -4.8122   0.9369
87   catalogu    19.008468   133.814873  Topic10  -5.5667   0.9505
129   gratuit    21.018616   214.352667  Topic10  -5.4662   0.5799
183       bel    14.822510   275.668074  Topic10  -5.8154  -0.0210

[510 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
478       1  0.170701      abord
478       4  0.814112      abord
602       6  0.977801      acces
651       6  0.950672  accessibl
741       3  0.936216     accord
...     ...       ...        ...
4         5  0.968857      épais
194       6  0.935481       étap
47        2  0.850091       état
47        9  0.144355       état
356       6  0.967212   éventuel

[762 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 3, 6, 10, 1, 9, 7, 5, 4])

In [33]:
# visualize results
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model2, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.169743  0.014320       1        1  18.537709
7     -0.129835  0.079088       2        1  16.616746
2     -0.145231 -0.115282       3        1   9.707569
5      0.133606 -0.035787       4        1   9.587086
9     -0.030056  0.188996       5        1   9.127113
0     -0.061006 -0.332002       6        1   8.619438
8     -0.153246 -0.026724       7        1   7.613367
6      0.235666  0.069779       8        1   7.393740
4     -0.028342  0.237044       9        1   7.302086
3      0.348188 -0.079432      10        1   5.495145, topic_info=         Term         Freq        Total Category  logprob  loglift
43        del  1294.000000  1294.000000  Default  30.0000  30.0000
64       bien  1723.000000  1723.000000  Default  29.0000  29.0000
5       envoi  1119.000000  1119.000000  Default  28.0000  28.0000
142      prix  1200.000000  1200.000000  Default  27.0000  27.0000
49      facil  1147.000000  1147.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
56    produir   176.003285  1209.794654  Topic10  -3.3418   0.9736
447       dat    40.451498   288.367206  Topic10  -4.8122   0.9372
87   catalogu    19.022952   133.710172  Topic10  -5.5667   0.9513
129   gratuit    21.034631   214.380501  Topic10  -5.4662   0.5797
183       bel    14.833804   275.680124  Topic10  -5.8154  -0.0210

[510 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
478       1  0.170745      abord
478       4  0.814320      abord
602       6  0.977800      acces
651       6  0.950670  accessibl
741       3  0.937136     accord
...     ...       ...        ...
4         5  0.968966      épais
194       6  0.935478       étap
47        2  0.850372       état
47        9  0.144403       état
356       6  0.967210   éventuel

[762 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 3, 6, 10, 1, 9, 7, 5, 4])

In [45]:
topics

['0.268*"prix" + 0.084*"rapport" + 0.029*"frais" + 0.028*"paiement" + 0.025*"linge" + 0.019*"meuble" + 0.019*"maison" + 0.018*"réduction" + 0.018*"correct" + 0.017*"port"',
 '0.061*"efficace" + 0.041*"housse" + 0.039*"couette" + 0.029*"draps" + 0.025*"belle" + 0.024*"satisfait" + 0.024*"redire" + 0.023*"acheté" + 0.021*"aussi" + 0.019*"sérieux"',
 '0.059*"redoute" + 0.029*"articles" + 0.020*"retour" + 0.019*"site" + 0.015*"trouve" + 0.015*"dommage" + 0.014*"vêtements" + 0.014*"pratique" + 0.012*"autres" + 0.011*"passé"',
 '0.184*"conforme" + 0.073*"rapidement" + 0.061*"expédition" + 0.059*"article" + 0.045*"description" + 0.038*"attentes" + 0.026*"ravie" + 0.025*"réception" + 0.024*"colis" + 0.022*"vite"',
 '0.066*"taille" + 0.039*"correspond" + 0.037*"couleur" + 0.029*"site" + 0.027*"commandé" + 0.027*"photo" + 0.024*"petit" + 0.020*"parfaite" + 0.020*"article" + 0.020*"agréable"',
 '0.077*"service" + 0.040*"temps" + 0.032*"client" + 0.031*"colis" + 0.028*"date" + 0.025*"redoute" + 0.